In [1]:
from cube import Cube
from tile import Tile
from alt_map import create_hex_map, valid_cubes

In [2]:
import random

In [3]:
random.seed(1945)
n_x = 129
n_y = 65
rgb_from_ijk = {cub.tuple():(random.randint(0,255), random.randint(0,255), random.randint(0,255)) for cub in valid_cubes(n_x,n_y)}

In [4]:
max_x = 10*(n_x*3-3)
max_y = 17*(n_y*2-2)

In [5]:
img = create_hex_map(rgb_from_ijk, max_x,max_y,n_x,n_y)
img.show()

In [6]:
pid_from_cube = {cub.tuple(): ind for ind, cub in enumerate(valid_cubes(n_x,n_y))}
cube_from_pid = {v: k for k, v in pid_from_cube.items()}
weight_from_pid = {pid: random.randint(1,8) for pid in cube_from_pid.keys()}

In [7]:
weight_from_cube = {Cube(cube_from_pid[k]): v for k,v in weight_from_pid.items()}

In [8]:
centers = random.sample(sorted(cube_from_pid.values()),40)
print(centers)
print(sum([sum([x == c for x in centers]) for c in centers]))

[(112, -78, -34), (29, -70, 41), (36, -53, 17), (25, -72, 47), (85, -90, 5), (94, -106, 12), (106, -110, 4), (38, -45, 7), (126, -116, -10), (64, -62, -2), (93, -79, -14), (31, -45, 14), (123, -123, 0), (26, -76, 50), (2, -40, 38), (98, -83, -15), (35, -49, 14), (82, -96, 14), (30, -72, 42), (69, -38, -31), (106, -113, 7), (47, -57, 10), (79, -95, 16), (124, -69, -55), (53, -80, 27), (26, -73, 47), (97, -92, -5), (15, -70, 55), (115, -83, -32), (20, -27, 7), (67, -93, 26), (33, -30, -3), (115, -73, -42), (42, -40, -2), (72, -57, -15), (43, -46, 3), (12, -53, 41), (116, -116, 0), (5, -37, 32), (73, -40, -33)]
40


In [9]:
def voronoi(centers, weight_from_cube):
    """Uses the domain of weight_from_cube to determine which cubes are eligible to be filled.
    - centers is a list of cubes
    - weight_from_cube is a dictionary of cube tuples to numbers
    returns a dictionary of cube tuples to the index of the centers list that they correspond to."""
    # distances = {cub: [0]*len(centers) if }
    centers = [Cube(x) for x in centers]  #If they're tuples, make them cubes.
    # See if any of the centers are on top of each other. TODO: Handle that case instead of just erroring.
    assert len(centers) == sum([sum([x == c for x in centers]) for c in centers])
    # Overall strategy: 
    # - seed all of the centers with distance 0
    # - add all neighbors of the centers to expand sets with the weight of the center as the 'distance'
    # - while there are any elements in the expansion sets, pick the lowest distance one and expand it
    # - if it's the lowest distance to the cell it expands to, add that to the expansion set
    # = finally, point each cell at its lowest distance source. 
    distmap = {Cube(k):{} for k in weight_from_cube.keys()}
    # Each center is in its own voronoi cell
    for cind, center in enumerate(centers):
        distmap[center][cind] = 0
        to_explore = set([center])
        # explored = set()
        while len(to_explore) > 0:
            cub = to_explore.pop()
            base_dist = distmap[cub][cind] + weight_from_cube[cub]
            expanding = [c for c in cub.neighbors() if c in weight_from_cube ]  # and c not in explored]
            for other in expanding:
                if len(distmap[other]) == 0 or min(distmap[other].values()) > base_dist:
                    distmap[other][cind] = base_dist
                    to_explore.add(other)
                else:
                    continue            
    #TODO: This currently is 'kind of fast' but could be faster. Two possible improvements:
    # Instead of starting with center 1 and fully calculating the distances, start with all centers simultaneously.
    # The previous, but instead of having to argmin the to_explore queue each time, have the queue split out by distance so that it's obvious where the next place to go is.
    result = dict()
    for cub in weight_from_cube.keys():
        dists = distmap.get(Cube(cub),{0:0})
        result[cub] = min(dists, key=dists.get)
    return result
        

In [10]:
def iterative_voronoi(num_centers, weight_from_cube, min_size, max_iters=20):
    """Given a set of weights, seed num_centers random centers and then keep going until all of the regions are at least min_size.
    Returns the pair of centers and ind_from_cube mapping."""
    assert num_centers * min_size < len(weight_from_cube)
    # assert num_centers * max_size > len(weight_from_cube)
    centers = random.sample(list(weight_from_cube.keys()),num_centers)
    guess = voronoi(centers, weight_from_cube)
    sizes = {c:0 for c in range(num_centers)}
    means = {c:Cube(0,0,0) for c in range(num_centers)}
    for cub, ind in guess.items():
        sizes[ind] += 1
        means[ind].add_in_place(cub)
    print(sum([min_size <= sizes[ind] for ind in range(num_centers)]))
    if all([min_size <= sizes[ind] for ind in range(num_centers)]):
        return centers, guess
    iter = 1
    while not all([min_size <= sizes[ind] for ind in range(num_centers)]):
        to_remove = []
        for ind in range(len(centers)):
            if min_size <= sizes[ind]:
                x = means[ind].x // sizes[ind]
                y = means[ind].y // sizes[ind]
                z = -x-y
                centers[ind] = Cube(x,y,z)
            elif sizes[ind] < min_size:
                to_remove.append(ind)
        argmaxes = sorted(sizes, key=sizes.get, reverse=True)
        for from_ind, to_ind in zip(to_remove, argmaxes):
            centers[from_ind] = random.sample([k for k,v in guess.items() if v==to_ind and k != centers[to_ind]], k=1)[0]
        guess = voronoi(centers, weight_from_cube)
        iter += 1
        if iter >= max_iters:
            return centers, guess
        sizes = {c:0 for c in range(num_centers)}
        means = {c:Cube(0,0,0) for c in range(num_centers)}
        for cub, ind in guess.items():
            sizes[ind] += 1
            means[ind].add_in_place(cub)
        print(sum([min_size <= sizes[ind] for ind in range(num_centers)]))
    return centers, guess

        



In [11]:
centers, result = iterative_voronoi(num_centers=80, weight_from_cube=weight_from_cube, min_size=61)
img = create_hex_map({cub.tuple(): rgb_from_ijk[centers[v]] for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()

59
69
72
72
73
72
73
74
73
70
71
71
70
73
72
76
77
78
77


KeyError: Cube(4, -7, 3)

In [12]:
img = create_hex_map({cub.tuple(): rgb_from_ijk[centers[v].tuple()] for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()

In [16]:
centers[1]

Cube(42, -72, 30)

In [17]:
img = create_hex_map({cub.tuple(): rgb_from_ijk[centers[v].tuple()] if v==1 else (0,0,0) for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()

In [65]:
img = create_hex_map({cub.tuple(): (v[1]//3,0,0) for cub, v in distmap.items()}, max_x,max_y,n_x,n_y)
img.show()

In [62]:
max([max([distmap[c][x] for c in weight_from_cube.keys()]) for x in range(len(centers))])

620

In [49]:
rgb_from_ijk[centers[29]]

(118, 191, 215)

In [28]:
def simple_voronoi(centers, weights_from_cube):
    """Uses the domain of weights_from_cube to determine which cubes are eligible to be filled.
    - centers is a list of cubes
    - weights_from_cube is a dictionary of cube tuples to numbers
    returns a dictionary of cube tuples to the index of the centers list that they correspond to."""
    # distances = {cub: [0]*len(centers) if }
    centers = [Cube(x) for x in centers]  #If they're tuples, make them cubes.
    # See if any of the centers are on top of each other. TODO: Handle that case instead of just erroring.
    assert len(centers) == sum([sum([x == c for x in centers]) for c in centers])
    result = {}
    for cub in weights_from_cube.keys():
        dists = {cind:Cube(cub).dist(c) for cind, c in enumerate(centers)}
        result[cub] = min(dists, key=dists.get)
    return result
        

In [29]:
result = simple_voronoi(centers, weight_from_cube)
img = create_hex_map({cub: rgb_from_ijk[centers[v]] for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()

In [26]:
img.show()

In [14]:
list(weight_from_cube.keys())[0]

(0, 0, 0)

In [20]:
{cub: rgb_from_ijk[centers[v]] for cub, v in result.items()}

{(0, 0, 0): (17, 107, 84),
 (0, -1, 1): (17, 107, 84),
 (0, -2, 2): (17, 107, 84),
 (0, -3, 3): (17, 107, 84),
 (0, -4, 4): (17, 107, 84),
 (0, -5, 5): (17, 107, 84),
 (0, -6, 6): (17, 107, 84),
 (0, -7, 7): (17, 107, 84),
 (0, -8, 8): (17, 107, 84),
 (0, -9, 9): (17, 107, 84),
 (0, -10, 10): (17, 107, 84),
 (0, -11, 11): (17, 107, 84),
 (0, -12, 12): (17, 107, 84),
 (0, -13, 13): (17, 107, 84),
 (0, -14, 14): (17, 107, 84),
 (0, -15, 15): (17, 107, 84),
 (0, -16, 16): (17, 107, 84),
 (0, -17, 17): (17, 107, 84),
 (0, -18, 18): (17, 107, 84),
 (0, -19, 19): (17, 107, 84),
 (0, -20, 20): (17, 107, 84),
 (0, -21, 21): (17, 107, 84),
 (0, -22, 22): (17, 107, 84),
 (0, -23, 23): (17, 107, 84),
 (0, -24, 24): (17, 107, 84),
 (0, -25, 25): (17, 107, 84),
 (0, -26, 26): (17, 107, 84),
 (0, -27, 27): (17, 107, 84),
 (0, -28, 28): (17, 107, 84),
 (0, -29, 29): (17, 107, 84),
 (0, -30, 30): (17, 107, 84),
 (0, -31, 31): (17, 107, 84),
 (0, -32, 32): (17, 107, 84),
 (0, -33, 33): (17, 107, 84),
 

In [21]:
result

{(0, 0, 0): 0,
 (0, -1, 1): 0,
 (0, -2, 2): 0,
 (0, -3, 3): 0,
 (0, -4, 4): 0,
 (0, -5, 5): 0,
 (0, -6, 6): 0,
 (0, -7, 7): 0,
 (0, -8, 8): 0,
 (0, -9, 9): 0,
 (0, -10, 10): 0,
 (0, -11, 11): 0,
 (0, -12, 12): 0,
 (0, -13, 13): 0,
 (0, -14, 14): 0,
 (0, -15, 15): 0,
 (0, -16, 16): 0,
 (0, -17, 17): 0,
 (0, -18, 18): 0,
 (0, -19, 19): 0,
 (0, -20, 20): 0,
 (0, -21, 21): 0,
 (0, -22, 22): 0,
 (0, -23, 23): 0,
 (0, -24, 24): 0,
 (0, -25, 25): 0,
 (0, -26, 26): 0,
 (0, -27, 27): 0,
 (0, -28, 28): 0,
 (0, -29, 29): 0,
 (0, -30, 30): 0,
 (0, -31, 31): 0,
 (0, -32, 32): 0,
 (0, -33, 33): 0,
 (0, -34, 34): 0,
 (0, -35, 35): 0,
 (0, -36, 36): 0,
 (0, -37, 37): 0,
 (0, -38, 38): 0,
 (0, -39, 39): 0,
 (0, -40, 40): 0,
 (0, -41, 41): 0,
 (0, -42, 42): 0,
 (0, -43, 43): 0,
 (0, -44, 44): 0,
 (0, -45, 45): 0,
 (0, -46, 46): 0,
 (0, -47, 47): 0,
 (0, -48, 48): 0,
 (0, -49, 49): 0,
 (0, -50, 50): 0,
 (0, -51, 51): 0,
 (0, -52, 52): 0,
 (0, -53, 53): 0,
 (0, -54, 54): 0,
 (0, -55, 55): 0,
 (0, -56, 56)